# <Center>  **Exploratory Data Analysis**

## Loading Data and Packages

First import all of the necessary Modules

In [2]:
# Import the Data Science Essentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
# Find and set the working directory for this project
os.chdir(r'C:\Users\jdrel\OneDrive\Documents\Data_Science\Springboard\Capstone-2')

Load the Data

In [4]:
# Import the data
data = pd.read_csv('./data/interim/KDD Data.csv')

In [6]:
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494020 entries, 0 to 494019
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   494020 non-null  int64  
 1   duration                     494020 non-null  int64  
 2   protocol_type                494020 non-null  object 
 3   service                      494020 non-null  object 
 4   flag                         494020 non-null  object 
 5   src_bytes                    494020 non-null  int64  
 6   dst_bytes                    494020 non-null  int64  
 7   land                         494020 non-null  int64  
 8   wrong_fragment               494020 non-null  int64  
 9   urgent                       494020 non-null  int64  
 10  hot                          494020 non-null  int64  
 11  num_failed_logins            494020 non-null  int64  
 12  logged_in                    494020 non-null  int64  
 13 

## Basic Analysis

This dataset has a large amount of independent variables that are hard to understand on their own. It's dependent variable is categorical with many categories that need some research to understand. The first thing to do is to research all of the categories of the dependent variable. To do this we can print out each of the types of network intrusion in the dataset in a format that can be used in markdown and ChatGPT

In [10]:
for type in data['labels'].unique():
    print(f"- {type}")

- normal.
- buffer_overflow.
- loadmodule.
- perl.
- neptune.
- smurf.
- guess_passwd.
- pod.
- teardrop.
- portsweep.
- ipsweep.
- land.
- ftp_write.
- back.
- imap.
- satan.
- phf.
- nmap.
- multihop.
- warezmaster.
- warezclient.
- spy.
- rootkit.


After plugging the above list into ChatGPT with the query "For each of these network intrusions can you please give a sentence summary describing what they are." It returned these explanations for each of the network intrusion types
- normal: This is not an attack type, but rather a classification used to indicate that the network traffic is within normal parameters.
- buffer_overflow: This attack occurs when a program or process attempts to store more data in a buffer than it was intended to hold, causing the excess data to overflow into adjacent memory locations.
- loadmodule: This type of attack involves the unauthorized loading of a module or plugin into a vulnerable system, which can then be used to gain further access or control over the system.
- perl: This is a type of attack that involves exploiting vulnerabilities in Perl scripts, which are often used to power web applications and other online services.
- neptune: This attack type involves overwhelming a targeted system with a flood of TCP packets, causing it to become unresponsive or crash.
- smurf: This attack type is similar to the neptune attack, but involves using a network of intermediary hosts to amplify the volume of traffic directed at the target system.
- guess_passwd: This attack type involves attempting to guess a user's password by systematically trying a large number of possible combinations.
- pod: This attack type involves sending malformed packets to a target system in order to cause it to crash or become unstable.
- teardrop: This is a type of attack that involves sending overlapping or malformed packets to a target system, causing it to crash or become unstable.
- portsweep: This attack involves scanning a range of network ports on a target system to identify potential vulnerabilities or open services.
- ipsweep: This attack involves scanning a range of IP addresses to identify potential targets for further attacks or exploitation.
- land: This attack type involves sending TCP packets with the source IP address set to the same as the destination IP address, causing the targeted system to become unresponsive or crash.
- ftp_write: This attack involves gaining unauthorized access to an FTP server and uploading or modifying files on the server.
- back: This type of attack involves installing backdoor software on a targeted system, which can then be used to gain unauthorized access or control over the system at a later time.
- imap: This attack involves exploiting vulnerabilities in the Internet Message Access Protocol (IMAP), which is commonly used for retrieving email messages.
- satan: This is a type of network scanning tool that can be used to identify potential vulnerabilities or open services on a target system.
- phf: This attack involves exploiting vulnerabilities in the Common Gateway Interface (CGI) scripts that are often used to power web applications and other online services.
- nmap: This is another type of network scanning tool that can be used to identify potential vulnerabilities or open services on a target system.
- multihop: This type of attack involves using a chain of intermediary hosts to mask the origin of the attack and make it more difficult to trace back to the attacker.
- warezmaster: This attack type involves using a compromised system to distribute pirated software or other illegal materials.
- warezclient: This attack involves using a compromised system to download pirated software or other illegal materials.
- spy: This type of attack involves monitoring or eavesdropping on network traffic in order to steal sensitive information or gain unauthorized access to a targeted system.
- rootkit: This is a type of malware that is designed to hide its presence on a targeted system, making it more difficult to detect or remove.

It seems as though some of these intrusions should clearly be grouped together. For instance portsweep and ipsweep are both types of sweeps that search for vulnerabilities in the system at different levels and are the same type of intrusion. But we can't be sure because it is possible that one of them would be more closely related with an actual attack since a type of sweep could lead to a ping of death (pod). Without clear information on how to group the different intrusions, we need to use cluster analysis to determine which groups look the closest. 
